# Actividad Integradora

## Descripción del problema

¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las cajas en algo parecido al orden y convertirlo en un negocio exitoso. 

Cada  robot  está  equipado  con  ruedas  omnidireccionales  y,  por  lo  tanto,  puede  conducir  en  las cuatro   direcciones.   Pueden   recoger   cajas   en   celdas   de   cuadrícula   adyacentes   con   sus manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es una pared, contiene una pila de cajas(y cuantas cajas hay en la pila)o está ocupado por otro robot. Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese momento. 

Lamentablemente, tu  presupuesto  resultó  insuficiente  para  adquirirun  software  de  gestión  de agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu tarea es enseñar a sus robots cómo ordenar su almacén. La organización delosagentesdepende de ti, siempre que todas las cajas terminen en pilas ordenadas de cinco.

Realiza la siguiente simulación:
* Inicializa las posiciones iniciales de las Kcajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
* Todos los agentes empiezan en posición aleatorias vacías.
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
* Número de movimientos realizados por todos los robots.
* Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.

In [ ]:
!pip3 install mesa
!pip3 install numpy
!pip3 install matplotlib

In [ ]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [ ]:
def obtener_almacen(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
    return habitacion
    
class Robot(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.caja = None
        self.movimientos = 0
        
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        
        # Defino el siguiente estado que va a tener el piso para la siguiente iteracion sin asignarlo todavia eso lo hago en el método `advance`.
        for vecino in vecinos:
            if isinstance(vecino, Caja):
                vecino.siguiente_stack_size = vecino.stack_size
                if vecino.siguiente_stack_size != 0 and vecino.siguiente_stack_size != 5:
                    if self.caja == None:
                        self.caja = vecino
                        vecino.siguiente_stack_size = 0
                    elif self.caja.stack_size + vecino.siguiente_stack_size <= 5:
                        self.caja.siguiente_stack_size = self.caja.stack_size + vecino.siguiente_stack_size
                        vecino.siguiente_stack_size = 0  
                    self.nueva_posicion = self.pos
                else:
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore=False,
                        include_center=False)
                    nueva_posicion = self.random.choice(vecindario)
                    self.nueva_posicion = nueva_posicion
        
    def advance(self):
        
        # Actualizamos el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        
        for vecino in vecinos:
                if isinstance(vecino, Caja) and vecino.pos == self.pos: 
                vecino.stack_size = vecino.siguiente_stack_size
                break
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            
        # Movemos la aspiradora a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)
    
class Caja(Agent):
    # Cantidad máxima de cajas 5
    
    def __init__(self, pos, modelo, size_stack=0):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.size_stack = size_stack
        self.siguiente_stack_size = None

class Almacen(Model):
    
    def __init__(self, m, n, num_agentes, num_cajas):
        self.num_agentes = num_agentes
        self.num_cajas = num_cajas
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        # Posicionar cajas de forma aleatoria
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in range(num_cajas):
            celda_vacia = random.choice(lista_celdas_vacias)
            caja = Caja(celda_vacia, self)
            caja.size_stack = 1
            self.grid.place_agent(caja, celda_vacia)
            self.schedule.add(caja)
            lista_celdas_vacias.remove(celda_vacia)
        
        # Posicionar celdas sin cajas
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            caja = Caja(celdas, self)
            self.grid.place_agent(caja, celdas)
            self.schedule.add(caja)
        
        # Posicionar agentes robots
        lista_celdas_sin_cajas = list(self.grid.empties)
        for i in range(num_agentes):
            celda_vacia = random.choice(lista_celdas_vacias)
            robot = Robot(i, self)
            self.grid.place_agent(robot, celda_vacia)
            self.schedule.add(robot)
            
        self.colectordatos = DataCollector(
            model_reporters={'Almacen': obtener_almacen},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todasceldaslimpias(self):
        celdas_ordenadas = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Caja) and contenido.size_stack == 5:
                    celdas_ordenadas += 1
        
        por_celdas_ordenadas = celdas_ordenadas / (self.grid.width * self.grid.height)
        if self.por_celdas_limpias == 1:
            return True
        else:
            return False

In [ ]:
# Datos de la habitacion:
M = 15
N = 15

# Numero de agentes
NUM_ROBOTS = 5
NUM_CAJAS = 10

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.1

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Almacen(M, N, NUM_AGENTES, NUM_CAJAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()
    #print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización

In [ ]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [ ]:
anim

In [ ]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias:', tiempo_ejecucion, '/', tiempo_inicio)
print('Porcentaje de celdas limpias después del termino de la simulación:', modelo.por_celdas_limpias)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())